In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, GlobalAveragePooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score # Import precision_score, recall_score, f1_score
from joblib import load
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Attention
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Attention, GlobalAveragePooling1D # Import GlobalAveragePooling1D

In [ ]:
model_path = "/content/bilstm_attention_model.keras"
print("Loading model...")
model = tf.keras.models.load_model(model_path)
print("Model loaded!")

Loading model...
Model loaded!


In [ ]:
#loading data and data pre-processing
train_file_path = '/content/train-00000-of-00001.parquet'
test_file_path = '/content/test-00000-of-00001.parquet'
train_df = pd.read_parquet(train_file_path, engine='pyarrow')
test_df = pd.read_parquet(test_file_path, engine='pyarrow')

X_train = train_df['sentence'].tolist()
y_train = train_df['relation'].tolist()
X_test = test_df['sentence'].tolist()
y_test = test_df['relation'].tolist()

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
class_weight_dict = dict(enumerate(class_weights))

glove_path = "/content/glove.6B.100d.txt"
embedding_dim = 100

word2vec = {}
with open(glove_path, 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vector


tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    vector = word2vec.get(word)
    if vector is not None:
        embedding_matrix[i] = vector


X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')
train_features = model.predict(X_train_pad)
test_features = model.predict(X_test_pad)



250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [ ]:
#Model evaluation
svm_model = SVC(kernel='linear', class_weight='balanced')
svm_model.fit(train_features, y_train_encoded)


y_pred = svm_model.predict(test_features)
accuracy = accuracy_score(y_test_encoded, y_pred)
precision = precision_score(y_test_encoded, y_pred, average='macro')
recall = recall_score(y_test_encoded, y_pred, average='macro')
f1 = f1_score(y_test_encoded, y_pred, average='macro')

print("Model Evaluation on Test Set:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nDetailed Classification Report:")
print(classification_report(y_test_encoded, y_pred))

Model Evaluation on Test Set:
Accuracy: 0.7118
Precision: 0.6537
Recall: 0.7025
F1 Score: 0.6725

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.91      0.85       134
           1       0.85      0.87      0.86       194
           2       0.68      0.75      0.71       162
           3       0.63      0.65      0.64       150
           4       0.73      0.92      0.81       153
           5       0.77      0.69      0.73        39
           6       0.83      0.84      0.83       291
           7       0.00      0.00      0.00         1
           8       0.74      0.84      0.79       211
           9       0.81      0.83      0.82        47
          10       0.45      0.45      0.45        22
          11       0.57      0.72      0.64       134
          12       0.56      0.62      0.59        32
          13       0.75      0.88      0.81       201
          14       0.74      0.87      0.80       210
     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

#  Load the trained model
model_path = "/content/bilstm_attention_model.keras"
print("Loading model...")
model = tf.keras.models.load_model(model_path)

# Load the tokenizer
tokenizer_path = "/content/tokenizer.pkl"
with open(tokenizer_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

#  Manually define relation ID mappings for SemEval 2010 Task 8
relation_id_to_label = {
          0: "Cause-Effect(e1,e2)",
          1: "Cause-Effect(e2,e1)",
          2: "Component-Whole(e1,e2)",
          3: "Component-Whole(e2,e1)",
          4: "Content-Container(e1,e2)",
          5: "Content-Container(e2,e1)",
          6: "Entity-Destination(e1,e2)",
          7: "Entity-Destination(e2,e1)",
          8: "Entity-Origin(e1,e2)",
          9: "Entity-Origin(e2,e1)",
          10: "Instrument-Agency(e1,e2)",
          11: "Instrument-Agency(e2,e1)",
          12: "Member-Collection(e1,e2)",
          13: "Member-Collection(e2,e1)",
          14: "Message-Topic(e1,e2)",
          15: "Message-Topic(e2,e1)",
          16: "Product-Producer(e1,e2)",
          17: "Product-Producer(e2,e1)",
          18: "Other"

}

#  Define function for relation extraction
def extract_relation_keras(sentence, model, tokenizer, relation_id_to_label, max_len=100):
    """
    Extracts the relation using the trained Keras model.

    Args:
        sentence (str): The input sentence.
        model (tf.keras.Model): The trained Keras model.
        tokenizer (Tokenizer): The tokenizer used for training.
        relation_id_to_label (dict): Dictionary mapping relation IDs to relation names.
        max_len (int): Maximum sequence length.

    Returns:
        tuple: (relation_id, relation_name)
    """
    # Tokenize and pad the sentence
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')

    # Predict relation
    features = model.predict(padded_sequence)
    predicted_relation_id = np.argmax(features, axis=1)[0]  # Get predicted class index

    # Retrieve the relation name
    predicted_relation_name = relation_id_to_label.get(predicted_relation_id, "Unknown Relation")

    return predicted_relation_id, predicted_relation_name

#  Real-time user input loop
print("\n Relation Extraction Model Ready! Type 'exit' to stop.")

while True:
    user_sentence = input("\nEnter a sentence:\n")

    if user_sentence.lower() == "exit":
        print("Exiting program...")
        break

    relation_id, relation_name = extract_relation_keras(user_sentence, model, tokenizer, relation_id_to_label)

    print(f" Predicted Relation ID: {relation_id}, Name: {relation_name}")

Loading model...

 Relation Extraction Model Ready! Type 'exit' to stop.

Enter a sentence:
The <e1>author</e1> of a keygen uses a <e2>disassembler</e2> to look at the raw assembly code.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
 Predicted Relation ID: 11, Name: Instrument-Agency(e2,e1)

Enter a sentence:
A misty <e1>ridge</e1> uprises from the <e2>surge</e2>.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
 Predicted Relation ID: 8, Name: Entity-Origin(e1,e2)

Enter a sentence:
The <e1>student</e1> <e2>association</e2> is the voice of the undergraduate student population of the State University of New York at Buffalo.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
 Predicted Relation ID: 12, Name: Member-Collection(e1,e2)

Enter a sentence:
exit
Exiting program...
